Task 1. Make subset of 200000 trajectories

In [ ]:
%matplotlib inline
import os
from collections import namedtuple
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pickle
import glob
PATH_TO_DATA = '/gpfs01/bethge/home/pmamidanna/deep_proprioception/data/'
PATH_TO_TEMPDATA = '/gpfs01/bethge/home/pmamidanna/deep_proprioception/data/unprocessed_data/'

files_list = glob.glob(os.path.join(PATH_TO_TEMPDATA, '*'))

In [ ]:
Latents = namedtuple('Latents', 'size rot shear_x shear_y speed noise')
all_jobs = [pickle.load(open(fname, 'rb')) for fname in files_list if 'vertical_inv' not in fname]

In [ ]:
all_datapoints = [datapoint for job in all_jobs for datapoint in job]

for datapoint in all_datapoints:
    datapoint['startpt'] = tuple(datapoint['startpt']) 

data = pd.DataFrame(all_datapoints)

subset = []
for label in range(1, 21):
    for plane in ['horizontal',  'vertical']:
        temp = data[data['label'] == label]
        temp = temp[temp['plane'] == plane]
        temp = temp.sort_values('muscle_jerk')
        subset.append(temp.iloc[:5050])

subset = pd.concat(subset)
subset[['size', 'rot', 'shear_x', 'shear_y', 'speed', 'noise']] = subset['latents'].apply(pd.Series)
subset = subset.drop(columns='latents')
subset = subset.drop_duplicates(
    subset=['label', 'startpt', 'size', 'rot', 'shear_x', 'shear_y', 'speed', 'noise', 'plane'])

In [ ]:
endeff = subset['endeffector_coords'] .tolist()
joint = subset['joint_coords'].tolist()
muscle = subset['muscle_coords'].tolist()

labels = subset['label'].tolist()
plane = subset['plane'].tolist()
startpt = subset['startpt'].tolist()
size = subset['size'].tolist()
rot = subset['rot'].tolist()
shear_x = subset['shear_x'].tolist()
shear_y = subset['shear_y'].tolist()
speed = subset['speed'].tolist()
noise = subset['noise'].tolist()

In [ ]:
def pad_nan(traj, max_length):
    """Pad nans to character trajectories that take less time."""
    num_dim = traj.shape[0]
    new_traj = np.zeros((num_dim, max_length))
    new_traj[:, :traj.shape[1]] = traj
    new_traj[:, traj.shape[1]:] = np.nan
    return new_traj

endeff = np.array([pad_nan(traj, 295) for traj in endeff])
joint = np.array([pad_nan(traj, 295) for traj in joint])
muscle = np.array([pad_nan(traj, 295) for traj in muscle])

In [ ]:
import h5py
with h5py.File('./data/pcr_full_temp.hdf5', 'w') as myfile:
    myfile.create_dataset('endeffector_coords', data=endeff)
    myfile.create_dataset('joint_coords', data=joint)
    myfile.create_dataset('muscle_coords', data=muscle)
    myfile.create_dataset('label', data=labels)
    myfile.create_dataset('plane', data=np.array(plane, dtype='S'))
    myfile.create_dataset('startpt', data=startpt)
    myfile.create_dataset('size', data=size)
    myfile.create_dataset('rot', data=rot)
    myfile.create_dataset('shear_x', data=shear_x)
    myfile.create_dataset('shear_y', data=shear_y)
    myfile.create_dataset('speed', data=speed)
    myfile.create_dataset('noise', data=noise)

In [ ]:
# Plot muscle jerks classwise!
import seaborn as sns
plt.figure(figsize=[14, 5])
sns.boxplot(x='label', y='muscle_jerk', hue='plane', data=subset)
charlabels = ['a', 'b', 'c', 'd', 'e', 'g', 'h', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 'u', 'v', 'w', 'y', 'z']
plt.xticks(np.arange(20), charlabels)
sns.despine(trim=True)

Task 2. Generate Spindle trajectories

**Rearranging muscles.**  

Date : 30th November.  

Previously, the arrangement of the muscles in the muscle configurations were according to alphabetical order. Now, I will convert into an order that is still alphabetical, but arranges shoulder muscles and then elbow muscles.

In [ ]:
import h5py
import numpy as np
import pandas as pd

with h5py.File('./data/pcr_full_temp.hdf5', 'r') as f:
    endeffector_coords = f['endeffector_coords'][()]
    joint_coords = f['joint_coords'][()]
    muscle_coords = f['muscle_coords'][()]
    labels = f['label'][()]
    planes = f['plane'][()]
    startpts = f['startpt'][()]
    size = f['size'][()]
    rot = f['rot'][()]
    shear_x = f['shear_x'][()]
    shear_y = f['shear_y'][()]
    speed = f['speed'][()]
    noise = f['noise'][()]

In [ ]:
old_order = ['ANC', 'BIClong', 'BICshort', 'BRA', 'BRD', 'CORB', 'DELT1', 'DELT2', 'DELT3',
             'ECRL', 'INFSP', 'LAT1', 'LAT2', 'LAT3', 'PECM1', 'PECM2', 'PECM3', 'PT', 'SUBSC', 
             'SUPSP','TMAJ', 'TMIN', 'TRIlat', 'TRIlong', 'TRImed']
new_order = ['CORB', 'DELT1', 'DELT2', 'DELT3', 'INFSP', 'LAT1', 'LAT2', 'LAT3', 'PECM1',
             'PECM2', 'PECM3', 'SUBSC', 'SUPSP', 'TMAJ', 'TMIN', 'ANC', 'BIClong', 'BICshort',
             'BRA', 'BRD', 'ECRL', 'PT', 'TRIlat', 'TRIlong', 'TRImed']

new_to_old = np.argsort(new_order)
old_to_new = np.argsort(new_to_old)

In [ ]:
new_muscle_coords = np.zeros_like(muscle_coords)
for i in range(new_muscle_coords.shape[0]):
    new_muscle_coords[i] = muscle_coords[i] if planes[i] == b'vertical' else muscle_coords[i, old_to_new, :]

In [ ]:
# Test!!

import opensim as osim
PATH_TO_OSIM_MODEL = '/gpfs01/bethge/home/pmamidanna/deep_proprioception/data/dynamic_arm_model/'
model = osim.Model(os.path.join(PATH_TO_OSIM_MODEL, 'MoBL_ARMS_module5_scaleIK.osim'))

import sys
sys.path.append('/gpfs01/bethge/home/pmamidanna/deep_proprioception/code/')
from pcr_data_utils import make_muscle_config

rand_idx = np.random.permutation(joint_coords.shape[0])[:10]

sample_joint_coords = [joint_coords[rand_id][:, np.all(~np.isnan(joint_coords[rand_id]), axis=0)]
                       for rand_id in rand_idx]
sample_muscle_coords_old = [muscle_coords[rand_id][:, np.all(~np.isnan(muscle_coords[rand_id]), axis=0)]
                            for rand_id in rand_idx]
sample_muscle_coords_new = [new_muscle_coords[rand_id][:, np.all(~np.isnan(new_muscle_coords[rand_id]), axis=0)]
                            for rand_id in rand_idx]

test_shuffling = []
for i in range(10):
    test_shuffling.append(make_muscle_config(model, sample_joint_coords[i]))

print('Difference between regenerated test samples and old muscle coordinates.')
print([np.linalg.norm(sample_muscle_coords_old[i] - test_shuffling[i]) for i in range(10)])
print('Difference between regenerated test samples and new muscle coordinates')
print([np.linalg.norm(sample_muscle_coords_new[i] - test_shuffling[i]) for i in range(10)])

**Compute fiber lengths instead of normalized fiber lengths.**  

Date: 6th December 2018

Look at the notebook `OptimalFiberLength` for more details.

In [ ]:
def collect_optimalfiberlengths(model):
    init_state = model.initSystem()
    model.equilibrateMuscles(init_state)

    # Prepare for simulation
    muscle_set = model.getMuscles() # returns a Set<Muscles> object
    num_muscles = muscle_set.getSize()

    optimalfiberlengths = {}
    for i in range(num_muscles):
        optimalfiberlengths[muscle_set.get(i).getName()] = muscle_set.get(i).getOptimalFiberLength()

    return optimalfiberlengths

optimal_fiber_lengths = collect_optimalfiberlengths(model)
optimal_length_multiplier = np.zeros(len(new_order))
for i in range(len(new_order)):
    optimal_length_multiplier[i] = optimal_fiber_lengths[new_order[i]]

In [ ]:
new_muscle_coords *= optimal_length_multiplier[None, :, None]
new_muscle_coords *= 1000 # Convert muscle length from m to mm.

**Compute Spindle Responses using the Prochazka Gorassini 1998 Model.**

In [ ]:
def signpow(a,b): return np.sign(a)*(np.abs(a)**b)

def make_spindle_coords(muscle_traj):
    stretch = np.gradient(muscle_traj, 1, axis=1)
    stretch_vel = np.gradient(muscle_traj, 0.015, axis=1)
    p_rate = 2*stretch + 4.3*signpow(stretch_vel, 0.6)
    return p_rate

def start_end_choice(traj):
    true_traj = traj[:, np.all(~np.isnan(traj), axis=0)]
    room = 320 - true_traj.shape[1]
    start_idx = np.random.randint(room)
    end_idx = start_idx + true_traj.shape[1]
    return start_idx, end_idx

def apply_shuffle(traj, start_idx, end_idx):
    true_traj = traj[:, np.all(~np.isnan(traj), axis=0)]
    mytraj = np.zeros((true_traj.shape[0], 320))
    mytraj[:, start_idx:end_idx] = true_traj
    mytraj[:, :start_idx] = true_traj[:, 0][:, None]
    mytraj[:, end_idx:] = true_traj[:, -1][:, None]
    return mytraj

def add_noise(mconf, factor):
    noisy_mconf = mconf + factor*mconf.std(axis=1)[:, None]*np.random.randn(*mconf.shape)
    return noisy_mconf

In [ ]:
nsamples = endeffector_coords.shape[0]
endeffector_coords_new = np.zeros((nsamples, 3, 320))
muscle_coords_new = np.zeros((nsamples, 25, 320))
joint_coords_new = np.zeros((nsamples, 4, 320))

In [ ]:
start_id = []
end_id = []
for i in range(nsamples):
    start_idx, end_idx = start_end_choice(endeffector_coords[i])
    endeffector_coords_new[i] = apply_shuffle(endeffector_coords[i], start_idx, end_idx)
    joint_coords_new[i] = apply_shuffle(joint_coords[i], start_idx, end_idx)
    muscle_coords_new[i] = apply_shuffle(new_muscle_coords[i], start_idx, end_idx)
    start_id.append(start_idx); end_id.append(end_idx)

In [ ]:
spindle_firing = np.zeros_like(muscle_coords_new)
for i in range(nsamples):
    temp = make_spindle_coords(muscle_coords_new[i])
    spindle_firing[i] = add_noise(temp, 0.3)

**Save Datasets!**

In [ ]:
# Make train and test splits.
num_examples = spindle_firing.shape[0]
shuffle_idx = np.random.RandomState(seed=12).permutation(num_examples)
start_id = np.array(start_id)
end_id = np.array(end_id)
num_train = int(0.8*200000)

In [ ]:
with h5py.File('./data/pcr_dataset_train.hdf5', 'w') as myfile:
    myfile.create_dataset('endeffector_coords', data=endeffector_coords_new[shuffle_idx[:num_train]])
    myfile.create_dataset('joint_coords', data=joint_coords_new[shuffle_idx[:num_train]])
    myfile.create_dataset('muscle_coords', data=muscle_coords_new[shuffle_idx[:num_train]])
    myfile.create_dataset('spindle_firing', data=spindle_firing[shuffle_idx[:num_train]])
    myfile.create_dataset('label', data=labels[shuffle_idx[:num_train]])
    myfile.create_dataset('plane', data=planes[shuffle_idx[:num_train]])
    myfile.create_dataset('startpt', data=startpts[shuffle_idx[:num_train]])
    myfile.create_dataset('size', data=size[shuffle_idx[:num_train]])
    myfile.create_dataset('rot', data=rot[shuffle_idx[:num_train]])
    myfile.create_dataset('shear_x', data=shear_x[shuffle_idx[:num_train]])
    myfile.create_dataset('shear_y', data=shear_y[shuffle_idx[:num_train]])
    myfile.create_dataset('speed', data=speed[shuffle_idx[:num_train]])
    myfile.create_dataset('start_id', data=start_id[shuffle_idx[:num_train]])
    myfile.create_dataset('end_id', data=end_id[shuffle_idx[:num_train]])

In [ ]:
with h5py.File('./data/pcr_dataset_test.hdf5', 'w') as myfile:
    myfile.create_dataset('endeffector_coords', data=endeffector_coords_new[shuffle_idx[num_train:200000]])
    myfile.create_dataset('joint_coords', data=joint_coords_new[shuffle_idx[num_train:200000]])
    myfile.create_dataset('muscle_coords', data=muscle_coords_new[shuffle_idx[num_train:200000]])
    myfile.create_dataset('spindle_firing', data=spindle_firing[shuffle_idx[num_train:200000]])
    myfile.create_dataset('label', data=labels[shuffle_idx[num_train:200000]])
    myfile.create_dataset('plane', data=planes[shuffle_idx[num_train:200000]])
    myfile.create_dataset('startpt', data=startpts[shuffle_idx[num_train:200000]])
    myfile.create_dataset('size', data=size[shuffle_idx[num_train:200000]])
    myfile.create_dataset('rot', data=rot[shuffle_idx[num_train:200000]])
    myfile.create_dataset('shear_x', data=shear_x[shuffle_idx[num_train:200000]])
    myfile.create_dataset('shear_y', data=shear_y[shuffle_idx[num_train:200000]])
    myfile.create_dataset('speed', data=speed[shuffle_idx[num_train:200000]])
    myfile.create_dataset('start_id', data=start_id[shuffle_idx[num_train:200000]])
    myfile.create_dataset('end_id', data=end_id[shuffle_idx[num_train:200000]])

In [ ]:
with h5py.File('./data/pcr_dataset_diy.hdf5', 'w') as myfile:
    myfile.create_dataset('endeffector_coords', data=endeffector_coords_new[shuffle_idx[:200000]], compression=9)
    myfile.create_dataset('label', data=labels[shuffle_idx[:200000]])
    myfile.create_dataset('plane', data=planes[shuffle_idx[:200000]])
    myfile.create_dataset('startpt', data=startpts[shuffle_idx[:200000]])
    myfile.create_dataset('size', data=size[shuffle_idx[:200000]])
    myfile.create_dataset('rot', data=rot[shuffle_idx[:200000]])
    myfile.create_dataset('shear_x', data=shear_x[shuffle_idx[:200000]])
    myfile.create_dataset('shear_y', data=shear_y[shuffle_idx[:200000]])
    myfile.create_dataset('speed', data=speed[shuffle_idx[:200000]])
    myfile.create_dataset('noise', data=noise[shuffle_idx[:200000]])
    myfile.create_dataset('start_id', data=start_id[shuffle_idx[:200000]])
    myfile.create_dataset('end_id', data=end_id[shuffle_idx[:200000]])